#Import


In [0]:
import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torch.nn.functional as F
import torchvision.utils as vutils
from PIL import ImageFile
from zipfile import ZipFile
import os

#Global Variables

In [0]:
batchSize = 1
img_size = 128
n_channel = 3 #Grey
n_encode = 64

n_disc = 16 #discriminator base output
n_gen = 64 # generator base output 

n_z = 50 #Latent vector size              
n_l = 10 #label number 
n_age = int(n_z/n_l) #5 ??? age category
n_gender = int(n_z/2) #25 ??? gender ?????

use_cuda = torch.cuda.is_available()
print(use_cuda)

des_dir="./hackathon_dataset_new"

True


#Connect to drive

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Extract data


In [0]:
#file_name = "./drive/My Drive/Research Project/hackathon_dataset_new.zip"
file_name2 = "./drive/My Drive/Research Project/Training/train_11.zip"
file_name3 = "./drive/My Drive/Research Project/Training/train_15.zip"

with ZipFile(file_name2, 'r') as zip:
  zip.extractall()
  print("Unzipped file")

with ZipFile(file_name3, 'r') as zip:
  zip.extractall()
  print("Unzipped file")
  
#with ZipFile(file_name2, 'r') as zip:
 # zip.extractall()
  #print("Unzipped file2")


Unzipped file
Unzipped file


#Create op directory

In [0]:
outf='./train_15'

if(os.path.exists(outf)):
  print("File exists")  
else:
  os.mkdir(outf)
  print("File created.")

"""
file_name2 = "./drive/My Drive/Training/train_7.zip"

with ZipFile(file_name2, 'r') as zip:
  zip.extractall()
  print("Unzipped file")
"""

File created.


'\nfile_name2 = "./drive/My Drive/Training/train_7.zip"\n\nwith ZipFile(file_name2, \'r\') as zip:\n  zip.extractall()\n  print("Unzipped file")\n'

#Dataloader

In [0]:
#load images
#!ls "/content/drive/My Drive/Research Project"

dataset = dset.ImageFolder(root=des_dir,
                           transform=transforms.Compose([transforms.Resize(128),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5,0.5),(0.5,0.5,0.5)),
                           ]))
#for i, j in dataset:
 # print(i,j) 
#dataset contains i=image tensor values, j = folder number 

#divide all images into batches of 20
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size= batchSize,
                                         shuffle=True, drop_last=True)
torch.save(dataloader, "%s/dataloader.pth"%(outf))

#dataloader=torch.load("./train_13/dataloader.pth")

for i,(j,k) in enumerate(dataloader):
      print(i,k)

#images/ batch size 20 =number of batches
#i=batch number, j=list of 20 tensor values, k=list of folder numbers


(0, tensor([8]))
(1, tensor([2]))
(2, tensor([13]))
(3, tensor([9]))
(4, tensor([9]))
(5, tensor([0]))
(6, tensor([11]))
(7, tensor([9]))
(8, tensor([15]))
(9, tensor([13]))
(10, tensor([6]))
(11, tensor([8]))
(12, tensor([2]))
(13, tensor([8]))
(14, tensor([4]))
(15, tensor([5]))
(16, tensor([8]))
(17, tensor([7]))
(18, tensor([15]))
(19, tensor([6]))
(20, tensor([8]))
(21, tensor([11]))
(22, tensor([9]))
(23, tensor([8]))
(24, tensor([19]))
(25, tensor([8]))
(26, tensor([3]))
(27, tensor([9]))
(28, tensor([15]))
(29, tensor([15]))
(30, tensor([8]))
(31, tensor([1]))
(32, tensor([14]))
(33, tensor([6]))
(34, tensor([15]))
(35, tensor([10]))
(36, tensor([8]))
(37, tensor([6]))
(38, tensor([6]))
(39, tensor([6]))
(40, tensor([18]))
(41, tensor([6]))
(42, tensor([6]))
(43, tensor([8]))
(44, tensor([9]))
(45, tensor([0]))
(46, tensor([10]))
(47, tensor([13]))
(48, tensor([15]))
(49, tensor([1]))
(50, tensor([1]))
(51, tensor([9]))
(52, tensor([13]))
(53, tensor([8]))
(54, tensor([11]))
(5

#One hot encoding

In [0]:
def one_hot(ageTensor):
    #print(ageTensor)
    oneHot = - torch.ones(batchSize*n_l).view(batchSize,n_l) #init all -1
    for i,j in enumerate(ageTensor): #batch size=20 ages
        oneHot[i,j] = 1 #in i=img number row only j=age column will be 1
    #print(oneHot)
    
    if use_cuda:
        return Variable(oneHot).cuda()
    else:
        return Variable(oneHot)

#Layers
Encoder (i) img data- conv2D, ReLU x4 (ii) Linear of (i)

Generator (i) Concatenate z,age,gender- linear, relu (ii) Conv2d transpose, relu x3 tanh of (i)

Dimg (i) img - conv2D (ii) concatenate(age,gender) - convtranspose + relu (iii) concatenate (i) and (ii) - conv2d + relu x3 (iv) Linear, relu of (iii) (v) Linear, sigmoid of (iv) (vi) Linear, softmax of (iv)

Dz linear + relu x3, sigmoid of z

#Encoder

In [0]:
class Encoder(nn.Module): #defining Encoder as a subclass of nn.Module
    def __init__(self): #runs initially #self=any arguments passed
        super(Encoder,self).__init__() #define super class
        self.layer1 = nn.Sequential( #feed-forward fully connected layers
            nn.Conv2d(n_channel,n_encode,5,2,2), #(input,output) where output=2Dconv(input) 
            #stride=5, padding,dilation=2
            nn.ReLU(),
            
            nn.Conv2d(n_encode,2*n_encode,5,2,2), #prev output becomes input
            nn.ReLU(),
            
            nn.Conv2d(2*n_encode,4*n_encode,5,2,2),
            nn.ReLU(),
            
            nn.Conv2d(4*n_encode,8*n_encode,5,2,2),
            nn.ReLU(),
        
        )
        self.layer2 = nn.Linear(8*n_encode*8*8,50) 
        #modeling the common structures shared by the input and output faces 
        #similar to the settings in J. Johnson et. al., “Perceptual losses for realtime style transfer and super-resolution”

        
    def forward(self,x):
        conv = self.layer1(x).view(-1,8*n_encode*8*8)
        out = self.layer2(conv)
        return out
print("done")

done


#Generator

In [0]:
class Generator(nn.Module): #(z, age_ohe, img_gender_v)
    def __init__(self):
        super(Generator,self).__init__()
        self.fc = nn.Sequential(nn.Linear(n_z+n_l*n_age+n_gender,#Creating a layer w linear transformation of args
                                          8*8*n_gen*16),# This FC layer collects all the features collected by breaking down inp image by previous multiple convolutions
                                  #together into an object that you want the network to recognize
                                nn.ReLU()) 
        
        self.upconv= nn.Sequential( #upsampling - transforms a small input to a large output
            nn.ConvTranspose2d(16*n_gen,8*n_gen,4,2,1),
            nn.ReLU(), #ConvTranspose upsamples with trainable kernels
            
            nn.ConvTranspose2d(8*n_gen,4*n_gen,4,2,1),
            nn.ReLU(),
            
            nn.ConvTranspose2d(4*n_gen,2*n_gen,4,2,1),
            nn.ReLU(),
            
            nn.ConvTranspose2d(2*n_gen,n_gen,4,2,1),
            nn.ReLU(),
            
            nn.ConvTranspose2d(n_gen,n_channel,3,1,1),
            nn.Tanh(),
        
        )
        
    def forward(self,z,age,gender):
        l = age.repeat(1,n_age) # (cols, rows) 1 x n_age matrix with age as every element 
        k = gender.view(-1,1).repeat(1,n_gender) # transpose gender and repeat
        z=z.float() # converting to float
        l=l.float()
        k=k.float()
        x = torch.cat([z,l,k],dim=1) #concatenate
        fc = self.fc(x).view(-1,16*n_gen,8,8) 
        out = self.upconv(fc)
        return out

#Dimg

In [0]:
class Dimg(nn.Module): #reconst,age_ohe.view(batchSize,n_l,1,1),img_gender_v.view(batchSize,1,1,1)
    def __init__(self):
        super(Dimg,self).__init__()
        self.conv_img = nn.Sequential(
            nn.Conv2d(n_channel,n_disc,4,2,1),
        )
        self.conv_l = nn.Sequential(
            nn.ConvTranspose2d(n_l*n_age+n_gender, n_l*n_age+n_gender, 64, 1, 0),
            nn.ReLU()
        )
        self.total_conv = nn.Sequential(
            nn.Conv2d(n_disc+n_l*n_age+n_gender,n_disc*2,4,2,1),
            nn.ReLU(),
            
            nn.Conv2d(n_disc*2,n_disc*4,4,2,1),
            nn.ReLU(),
            
            nn.Conv2d(n_disc*4,n_disc*8,4,2,1),
            nn.ReLU()
        )
        
        self.fc_common = nn.Sequential(
            nn.Linear(8*8*img_size,1024),
            nn.ReLU()
        )
        self.fc_head1 = nn.Sequential(
            nn.Linear(1024,1),
            nn.Sigmoid()
        )
        self.fc_head2 = nn.Sequential(
            nn.Linear(1024,n_l),
            nn.Softmax(dim=-2)
        )
        
    def forward(self,img,age,gender):
        l = age.repeat(1,n_age,1,1,)
        k = gender.repeat(1,n_gender,1,1,)
        conv_img = self.conv_img(img)
        conv_l   = self.conv_l(torch.cat([l,k],dim=1))
        catted   = torch.cat((conv_img,conv_l),dim=1)
        total_conv = self.total_conv(catted).view(-1,8*8*img_size)
        body = self.fc_common(total_conv)
        
        head1 = self.fc_head1(body)
        head2 = self.fc_head2(body)
        
        return head1,head2

#Dz

In [0]:
class Dz(nn.Module): #z(encoder) or z_star(prior)
    def __init__(self):
        super(Dz,self).__init__()
        self.model = nn.Sequential(
            nn.Linear(n_z,n_disc*4),
            nn.ReLU(),
            
            nn.Linear(n_disc*4,n_disc*2),
            nn.ReLU(),
            
            nn.Linear(n_disc*2,n_disc),
            nn.ReLU(),
            
            nn.Linear(n_disc,1),
            nn.Sigmoid()
        )
    def forward(self,z):
        return self.model(z)

#Losses

In [0]:
if use_cuda:
    BCE = nn.BCELoss().cuda() #Binary Cross entropy loss
    L1  = nn.L1Loss().cuda() #mean absolute error- sum of all differences between true and predicted value
    CE = nn.CrossEntropyLoss().cuda() #cross entropy Loss
    MSE = nn.MSELoss().cuda() #mean square error- sum of all sqaured differences between true and predicted value
else:
    BCE = nn.BCELoss()
    L1  = nn.L1Loss()
    CE = nn.CrossEntropyLoss()
    MSE = nn.MSELoss()
    
def TV_LOSS(imgTensor): #total variation regularization
    x = (imgTensor[:,:,1:,:]-imgTensor[:,:,:img_size-1,:])**2
    y = (imgTensor[:,:,:,1:]-imgTensor[:,:,:,:img_size-1])**2 
    out = (x.mean(dim=3)+y.mean(dim=2)).mean()
    return out

#Fixed_l

In [0]:
#doesnt need to be saved for [32,64] run because it is standard
fixed_l = -torch.ones(10*10).view(10,10) #initialize 80x10 matrix with value -1
for i,l in enumerate(fixed_l): #fix 1 for each of the 80 tensors
    l[i//1] = 1
fixed_l_v = Variable(fixed_l)
if use_cuda:
    fixed_l_v = fixed_l_v.cuda()

#Initialise nets

In [0]:
if use_cuda:
    netE = Encoder().cuda()
    netD_img = Dimg().cuda()
    netD_z  = Dz().cuda()
    netG = Generator().cuda()
else:
    netE = Encoder()
    netD_img = Dimg()
    netD_z  = Dz()
    netG = Generator()

#Initialise weights

In [0]:
def weights_init(m):  #weights calculated according to class name    
    classname = m.__class__.__name__
    if classname.find('Conv') != -1 or classname.find("Linear") !=-1: #if conv or linear
        m.weight.data.normal_(0.0, 0.02) #mean, standard deviation
    elif classname.find('BatchNorm') != -1: #batch normalization
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


netE.apply(weights_init)
netD_img.apply(weights_init)
netD_z.apply(weights_init)
netG.apply(weights_init)

Generator(
  (fc): Sequential(
    (0): Linear(in_features=125, out_features=65536, bias=True)
    (1): ReLU()
  )
  (upconv): Sequential(
    (0): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
    (4): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (5): ReLU()
    (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): ReLU()
    (8): ConvTranspose2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): Tanh()
  )
)

#Loading a model 

In [0]:
"""
#Encoder
#for param_tensor in netE.state_dict():
 #  print(param_tensor, "\t", netE.state_dict()[param_tensor].size())
 
checkpoint = torch.load("./train_13/netE.pth")
netE.load_state_dict(checkpoint)
netE.train()
netE.eval()

#Generator
checkpoint1 = torch.load("./train_13/netG.pth")
netG.load_state_dict(checkpoint1)
netG.train()
netG.eval()

#Dimg
checkpoint2 = torch.load("./train_13/netD_img.pth")
netD_img.load_state_dict(checkpoint2)
netD_img.train()
netD_img.eval()

#Dz
checkpoint3 = torch.load("./train_13/netD_z.pth")
netD_z.load_state_dict(checkpoint3)
netD_z.train()
netD_z.eval()

fixed_noise=torch.load("./train_13/fixed_noise.pth")
fixed_g=torch.load("./train_13/fixed_g.pth")
fixed_img_v = Variable(fixed_noise)
fixed_g_v = Variable(fixed_g)
            
if use_cuda:
   fixed_img_v = fixed_img_v.cuda()
   fixed_g_v = fixed_g_v.cuda()
"""

'\n#Encoder\n#for param_tensor in netE.state_dict():\n #  print(param_tensor, "\t", netE.state_dict()[param_tensor].size())\n \ncheckpoint = torch.load("./train_13/netE.pth")\nnetE.load_state_dict(checkpoint)\nnetE.train()\nnetE.eval()\n\n#Generator\ncheckpoint1 = torch.load("./train_13/netG.pth")\nnetG.load_state_dict(checkpoint1)\nnetG.train()\nnetG.eval()\n\n#Dimg\ncheckpoint2 = torch.load("./train_13/netD_img.pth")\nnetD_img.load_state_dict(checkpoint2)\nnetD_img.train()\nnetD_img.eval()\n\n#Dz\ncheckpoint3 = torch.load("./train_13/netD_z.pth")\nnetD_z.load_state_dict(checkpoint3)\nnetD_z.train()\nnetD_z.eval()\n\nfixed_noise=torch.load("./train_13/fixed_noise.pth")\nfixed_g=torch.load("./train_13/fixed_g.pth")\nfixed_img_v = Variable(fixed_noise)\nfixed_g_v = Variable(fixed_g)\n            \nif use_cuda:\n   fixed_img_v = fixed_img_v.cuda()\n   fixed_g_v = fixed_g_v.cuda()\n'

#Optimiser

In [0]:

#implementing Adam optimization algorithm 
optimizerE = optim.Adam(netE.parameters(),lr=0.0002,betas=(0.5,0.999))
optimizerD_z = optim.Adam(netD_z.parameters(),lr=0.0002,betas=(0.5,0.999))
optimizerD_img = optim.Adam(netD_img.parameters(),lr=0.0002,betas=(0.5,0.999))
optimizerG = optim.Adam(netG.parameters(),lr=0.0002,betas=(0.5,0.999))
#lr= learning rate
#betas= coefficients for averages
"""
checkpoint4 = torch.load("./train_13/optimizerE.pth")
optimizerE.load_state_dict(checkpoint4)
checkpoint5 = torch.load("./train_13/optimizerD_z.pth")
optimizerD_z.load_state_dict(checkpoint5)
checkpoint6 = torch.load("./train_13/optimizerD_img.pth")
optimizerD_img.load_state_dict(checkpoint6)
checkpoint7 = torch.load("./train_13/optimizerG.pth")
optimizerG.load_state_dict(checkpoint7)
"""

'\ncheckpoint4 = torch.load("./train_13/optimizerE.pth")\noptimizerE.load_state_dict(checkpoint4)\ncheckpoint5 = torch.load("./train_13/optimizerD_z.pth")\noptimizerD_z.load_state_dict(checkpoint5)\ncheckpoint6 = torch.load("./train_13/optimizerD_img.pth")\noptimizerD_img.load_state_dict(checkpoint6)\ncheckpoint7 = torch.load("./train_13/optimizerG.pth")\noptimizerG.load_state_dict(checkpoint7)\n'

#Main

In [0]:
for epoch in range(20):
  print(epoch, 'Epoch')
  for batch_num,(img_data,folder_list) in enumerate(dataloader):
      #get gender and age from label
      print(batch_num)
      img_gender=(folder_list%2)*2-1 #female=-1, male =1  
      img_age = (folder_list-(folder_list%2))*4
              
      #convert to variable
      img_data_v = Variable(img_data) 
      img_age_v = Variable(img_age).view(-1,1) #+transpose
      img_gender_v = Variable(img_gender.float())
      
      if use_cuda: #appending device=cuda to variable
            img_data_v = img_data_v.cuda()
            img_age_v = img_age_v.cuda()
            img_gender_v = img_gender_v.cuda()  
      
      if epoch==0 and batch_num==0: #initialize noise for data and gender by taking first 8 values in first loop and repeating
            fixed_noise = img_data[:1].repeat(10,1,1,1)
            fixed_g=img_gender[:1].view(-1,1).repeat(10,1)

            #needs to be saved for [32,64] run because we cant get the same values again
            torch.save(fixed_noise, "%s/fixed_noise.pth"%(outf))
            torch.save(fixed_g,"%s/fixed_g.pth"%(outf))
 
            fixed_img_v = Variable(fixed_noise)
            fixed_g_v = Variable(fixed_g)
            
            if use_cuda:
                 fixed_img_v = fixed_img_v.cuda()
                 fixed_g_v = fixed_g_v.cuda()
                  
      #one hot encoding of ages
      #print(img_age)
      img_age_ohe = (img_age)/8
      #print(img_age_ohe)            
      age_ohe = one_hot(img_age_ohe)
      
      #prior distribution assignment
      z_star = Variable(torch.FloatTensor(batchSize*n_z).uniform_(-1,1)).view(batchSize,n_z) #20x40 values between -1 to 1
      real_label = Variable(torch.ones(batchSize).fill_(1)).view(-1,1) #20 values = 1
      fake_label = Variable(torch.ones(batchSize).fill_(0)).view(-1,1) #20 values = 0

      if use_cuda:
            z_star, real_label, fake_label = z_star.cuda(),real_label.cuda(),fake_label.cuda()
          
      
      #encoder
      netE.zero_grad() #initializes gradients to zero, else by default pytorch accumulates gradients for each pas
      z = netE(img_data_v) #encode img data

      #generator
      netG.zero_grad()
      reconst = netG(z,age_ohe,img_gender_v) #generate using encoded img data, age, gender
      EG_loss1 = L1(reconst,img_data_v) #mean absolute error - comparing generator reconstr value with img data
    
      #discriminator applied to generated images
      D_reconst,_ = netD_img(reconst,age_ohe.view(batchSize,n_l,1,1),img_gender_v.view(batchSize,1,1,1))
      EG_loss2 = BCE(D_reconst,real_label) #binary cross entropy loss - comparing disc(gen) output with 1s
      #discriminator applied to encoded(img data)
      Dz = netD_z(z)
      EG_loss3 = BCE(Dz,real_label) #binary cross entropy loss - comparing disc(enc(img data)) with 1s

      #generator using z.detach()
      reconst = netG(z.detach(),age_ohe,img_gender_v) #detach - when backward propagation, it stops here and doesnt traverse to z
      EG_loss4 = TV_LOSS(reconst)
      
      #compute EG loss -> backward propagation -> optimization
      EG_loss = EG_loss1 + 0.0001*EG_loss2 + 0.01*EG_loss3 + EG_loss4
      EG_loss.backward()
      optimizerE.step()
      optimizerG.step()
        
      #train netD_z with prior distribution
      netD_z.zero_grad()
      Dz_prior = netD_z(z_star)
      
      Dz = netD_z(z.detach())
      
      #compute Dz losses -> backward propagation -> optimization
      Dz_loss = BCE(Dz_prior,real_label)+BCE(Dz,fake_label)      
      Dz_loss.backward()      
      optimizerD_z.step()

      # train D_img with real images
      netD_img.zero_grad()
      D_img,D_clf = netD_img(img_data_v,age_ohe.view(batchSize,n_l,1,1),img_gender_v.view(batchSize,1,1,1))
      D_reconst,_ = netD_img(reconst.detach(),age_ohe.view(batchSize,n_l,1,1),img_gender_v.view(batchSize,1,1,1)) #detach

      #compute D loss -> backward propagation -> optimization
      D_loss = BCE(D_img,real_label)+BCE(D_reconst,fake_label)
      D_loss.backward()
      optimizerD_img.step()
 
  #for every epoch after all batches       
  fixed_z = netE(fixed_img_v)
  fixed_fake = netG(fixed_z,fixed_l_v,fixed_g_v)
  vutils.save_image(fixed_fake.data,
                '%s/reconst_epoch%03d.png' % (outf,epoch+1),
                normalize=True) #save img
  ep=epoch+1       
  msg1 = "epoch:{}, step:{}".format(epoch+1,batch_num+1)
  msg2 = format("EG loss1:%f"%(EG_loss1.data.item()),"<30")+"|"+format("EG loss2:%f"%(EG_loss2.data.item()),"<30")
  msg5 = format("EG loss4:%f"%(EG_loss4.data.item()),"<30")+"|"+"EG_loss3:%f"%(EG_loss3.data.item())
  msg6 = format("EG loss:%f"%(EG_loss.data.item()),"<30")
  msg3 = format("D_img:%f"%(D_img.mean().data.item()),"<30")+"|"+format("D_reconst:%f"%(D_reconst.mean().data.item()),"<30")\
   +"|"+format("D_loss:%f"%(D_loss.data.item()),"<30")
  msg4 = format("D_z:%f"%(Dz.mean().data.item()),"<30")+"|"+format("D_z_prior:%f"%(Dz_prior.mean().data.item()),"<30")+"|"+format("Dz_loss:%f"%(Dz_loss.data.item()),"<30")
  f= open("./losses_15.txt","a+")
  msg= format(epoch+1)+" "+format("%f"%(EG_loss1.data.item()))+" "+format("%f"%(EG_loss2.data.item()))+" "+format("%f"%(EG_loss3.data.item()))+" "+format("%f"%(EG_loss4.data.item()))+" "+format("%f"%(EG_loss.data.item()))+" "+format("%f"%(D_img.mean().data.item()))+" "+format("%f"%(D_reconst.mean().data.item()))+" "+format("%f"%(D_loss.data.item()))+" "+format("%f"%(Dz.mean().data.item()))+" "+format("%f"%(Dz_prior.mean().data.item()))+" "+format("%f"%(Dz_loss.data.item()))
  f.write(msg+"\n")
  f.close()
  print("-"*80)
  print(msg1)
  print(msg2)
  print(msg5)
  print(msg6)
  print(msg3)
  print(msg4)       
  print("-"*80)
  #for every two epoch - checkpoints
  if epoch%2==1:
      torch.save(netE.state_dict(),"%s/netE.pth"%(outf))
      torch.save(netG.state_dict(),"%s/netG.pth"%(outf))
      torch.save(netD_img.state_dict(),"%s/netD_img.pth"%(outf))
      torch.save(netD_z.state_dict(),"%s/netD_z.pth"%(outf))

      #optimiser
      torch.save(optimizerE.state_dict(),"%s/optimizerE.pth"%(outf))
      torch.save(optimizerG.state_dict(),"%s/optimizerG.pth"%(outf))
      torch.save(optimizerD_img.state_dict(),"%s/optimizerD_img.pth"%(outf))
      torch.save(optimizerD_z.state_dict(),"%s/optimizerD_z.pth"%(outf))

      !zip -r ./train_15.zip ./train_15
      from google.colab import drive
      drive.mount('/content/drive')
      !cp "train_15.zip" "/content/drive/My Drive/Research Project/Training/train_15.zip"
      !cp "losses_15.txt" "/content/drive/My Drive/Research Project/Losses/losses_15.txt"

(0, 'Epoch')
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
2

In [0]:
#!ls /content
#!rm -rf /content

#Download output folder


In [0]:
#!zip -r ./result_tv_gender.zip ./result_tv_gender
!zip -r ./train_15.zip ./train_15

updating: train_15/ (stored 0%)
updating: train_15/netD_img.pth (deflated 7%)
updating: train_15/netG.pth (deflated 7%)
updating: train_15/optimizerE.pth (deflated 82%)
updating: train_15/optimizerD_z.pth (deflated 11%)
updating: train_15/optimizerG.pth (deflated 51%)
updating: train_15/dataloader.pth (deflated 81%)
updating: train_15/optimizerD_img.pth (deflated 65%)
updating: train_15/netE.pth (deflated 7%)
updating: train_15/reconst_epoch001.png (deflated 1%)
updating: train_15/netD_z.pth (deflated 8%)
updating: train_15/fixed_g.pth (deflated 33%)
updating: train_15/fixed_noise.pth (deflated 75%)
updating: train_15/reconst_epoch002.png (deflated 1%)
updating: train_15/reconst_epoch003.png (deflated 1%)
updating: train_15/reconst_epoch004.png (deflated 1%)
updating: train_15/reconst_epoch006.png (deflated 1%)
updating: train_15/reconst_epoch005.png (deflated 1%)
updating: train_15/reconst_epoch007.png (deflated 1%)
updating: train_15/reconst_epoch008.png (deflated 1%)
updating: train

In [0]:
#!ls "/content/drive/My Drive"
#!rmdir --help

In [0]:
#!cp "result_tv_gender.zip" "/content/drive/My Drive/result_tv_gender.zip"
!cp "train_15.zip" "/content/drive/My Drive/Research Project/Training/train_15.zip"